In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

from logger import Logger
from solver import Solver
from crossngover import CrossN

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# TEST LOGGER

In [3]:
experiment_note = """ 
NET: Mobilenet 
Expereiment goal: debugging 
Training mode: Normal Evo + Crossn
Params: Default"""

logger = Logger('debug_01', experiment_note)


./2019-11-17/debug_01_16:55 was created


In [4]:
post_note = "Test After Experiment info"
logger.add_post_result(post_note)

In [9]:
net = torchvision.models.mobilenet_v2(pretrained=True)
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
evo_optim = CrossN()


def mse(yh, h):
    return ((yh - h)**2).sum()

In [10]:
solver = Solver(
    net,
    optimizer,
    logger,
    criterion,
    mse,
    evo_optim, 
    trainloader,
    testloader,
    testloader)

In [11]:
solver.start()

Start training


TypeError: unsupported operand type(s) for -: 'MobileNetV2' and 'DataLoader'